In [ ]:
import os
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

plt.rc('font', **{'family': 'sans-serif', 'size': 10})

In [ ]:
future = 0.5  # [hours]
# pkl_files = sorted(glob.glob(f'*{future:.2f}_hours-consumption.pkl'))
pkl_files = sorted(glob.glob(f'*{future:.2f}_hours-consumption-generation.pkl'))
suffix = '-'.join(os.path.splitext(pkl_files[0])[0].split('-')[1:])
data = [pickle.load(open(pkl_file, 'rb')) for pkl_file in pkl_files]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,3))
stop = 4
ylim = [0.8, 8.2]
for i,(today,tomorrow) in enumerate(zip(data[:-1], data[1:])):
    ax.plot(i * 24 + np.zeros(2), ylim, '--', lw=1, color=[.6,.6,.6])
    if i == 0:
        ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=1, label='Measured')
        ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=1, label='Predicted')
    else:
        ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=1)
        ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=1)
    mape = 100 * np.mean(np.abs((tomorrow['past_measured'] - today['future_predicted']) / tomorrow['past_measured']))
    print(f'MAPE = {mape:.2f}%')
    if i >= stop:
        break
i += 1
ax.plot(i * 24 + np.zeros(2), ylim, '--', lw=1, color=[.6,.6,.6])
if i <= stop:
    ax.plot((i+1) * 24 + np.zeros(2), ylim, '--', lw=1, color=[.6,.6,.6])
    ax.plot(tomorrow['t_past'] + i * 24, tomorrow['past_measured'], 'k', lw=1)
    ax.plot(tomorrow['t_future'] + i * 24, tomorrow['future_predicted'], 'r', lw=1)
# ax.legend(loc='upper left')
ax.set_xticks(np.r_[-24 : 121 : 12])
ax.set_xlabel('Time [hours]')
ax.set_ylabel('Consumption [MW]')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.grid(which='major', axis='y', color=[.6,.6,.6], lw=0.5, linestyle=':')
ax.set_xlim([-26, 122])
ax.set_ylim(ylim)
ax.set_yticks(np.r_[1 : 8.5])
fig.tight_layout()
fig.savefig(f'prediction_{suffix}_{stop+1}.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(8,4))
for i,today in enumerate(data):
    if i == 0:
        ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=1, label='Measured')
        ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=1, label='Predicted')
    else:
        ax.plot(today['t_past'] + i * 24, today['past_measured'], 'k', lw=1)
        ax.plot(today['t_future'] + i * 24, today['future_predicted'], 'r', lw=1)
ax.legend(loc='upper left')
ax.set_xticks(np.r_[-24 : 121 : 12])
ax.set_xlabel('Time [hours]')
ax.set_ylabel('Consumption [MW]')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.grid(which='major', axis='y', color=[.6,.6,.6], lw=0.5, linestyle=':')
ax.set_xlim([-26, 122])
fig.tight_layout()